## Chapter Exercises

### Exercise 1

Determine if a valid `Functor` can be written for the datatype provided.


1.
```haskell
data Bool =
    False | True
```

<font color="green">**Answer:**</font> No this type has kind `*`.

2.
```haskell
data BoolAndSomethingElse a =
    False' a | True' a
```

<font color="green">**Answer:**</font> Yes.

3.
```haskell
data BoolAndMaybeSomethingElse a =
    Falsish | Truish a
```

<font color="green">**Answer:**</font> Yes.

4. Use the kinds to guide you on this one, don’t get too hung up on the details.
```haskell
newtype Mu f = InF { outF :: f (Mu f) }
```

<font color="green">**Answer:**</font> No this type has kind `(* -> *) -> *`, we can not apply a type to it in such a way that it has kind `* -> *`.

5. Again, follow the kinds and ignore the unfamiliar parts
```haskell
import GHC.Arr

data D =
    D (Array Word Word) Int Int
```

<font color="green">**Answer:**</font> No this type has kind `*`.

### Exercise 2

Rearrange the arguments to the type constructor of the datatype so the Functor instance works.

```haskell
1. 

data Sum a b =
    First a
  | Second b

instance Functor (Sum e) where
    fmap f (First a) = First (f a)
    fmap f (Second b) = Second b
```

In [3]:
data Sum a b =
    First a
  | Second b

instance Functor (Sum e) where
    fmap _ (First a) = First a
    fmap f (Second b) = Second (f b)

2. 

```haskell
data Company a b c =
    DeepBlue a c
  | Something b

instance Functor (Company e e') where
    fmap f (Something b) = Something (f b)
    fmap _ (DeepBlue a c) = DeepBlue a c
```

In [5]:
data Company a b c =
    DeepBlue a c
  | Something b

instance Functor (Company e e') where
    fmap _ (Something b) = Something b
    fmap f (DeepBlue a c) = DeepBlue a (f c)

3. 
```haskell
data More a b =
    L a b a
  | R b a b
  deriving (Eq, Show)

instance Functor (More x) where
    fmap f (L a b a') = L (f a) b (f a')
    fmap f (R b a b') = R b (f a) b'
```
Keeping in mind that it should result in a Functor that does the following:
```
Prelude> fmap (+1) (L 1 2 3)
L 2 2 4

Prelude> fmap (+1) (R 1 2 3)
R 1 3 3
```



In [7]:
data More b a =
    L a b a
  | R b a b
  deriving (Eq, Show)

instance Functor (More x) where
    fmap f (L a b a') = L (f a) b (f a')
    fmap f (R b a b') = R b (f a) b'

### Exercise 3

Write Functor instances for the following datatypes.

1.
```haskell
data Quant a b =
    Finance
  | Desk a
  | Bloor b
```

In [8]:
data Quant a b =
    Finance
  | Desk a
  | Bloor b


instance Functor (Quant a) where
    fmap _ Finance = Finance
    fmap _ (Desk x) = Desk x
    fmap f (Bloor y) = Bloor (f y)

2. No, it’s not interesting by itself.
```haskell
data K a b =
    K a
```

In [17]:
data K a b =
    K a

instance Functor (K a) where
    fmap f (K x) = K x

Line 1: Use newtype instead of data
Found:
data K a b = K a
Why not:
newtype K a b = K a

3.
```haskell
{-# LANGUAGE FlexibleInstances #-}
newtype Flip f a b =
    Flip (f b a)
    deriving (Eq, Show)

newtype K a b =
    K a

-- should remind you of an
-- instance you've written before
instance Functor (Flip K a) where
    fmap = undefined
```

In [21]:
newtype Flip f a b =
    Flip (f b a)
    deriving (Eq, Show)

newtype K a b =
    K a

-- should remind you of an
-- instance you've written before
instance Functor (Flip K a) where
    fmap f (Flip (K x)) = Flip (K $ f x)

4.
```haskell
data EvilGoateeConst a b =
    GoatyConst b
-- You thought you'd escaped the goats
-- by now didn't you? Nope.
```
No, it doesn’t do anything interesting. No magic here or in the
previous exercise. If it works, you succeeded.

In [22]:
data EvilGoateeConst a b =
    GoatyConst b
    
instance Functor (EvilGoateeConst a) where
    fmap f (GoatyConst x) = GoatyConst $ f x

Line 1: Use newtype instead of data
Found:
data EvilGoateeConst a b = GoatyConst b
Why not:
newtype EvilGoateeConst a b = GoatyConst b

5. Do you need something extra to make the instance work?
```haskell
data LiftItOut f a =
    LiftItOut (f a)
```

In [24]:
data LiftItOut f a =
    LiftItOut (f a)

instance Functor f => Functor (LiftItOut f) where
    fmap f (LiftItOut fa) = LiftItOut (fmap f fa)

Line 1: Use newtype instead of data
Found:
data LiftItOut f a = LiftItOut (f a)
Why not:
newtype LiftItOut f a = LiftItOut (f a)

6.
```haskell
data Parappa f g a =
    DaWrappa (f a) (g a)
```

In [30]:
data Parappa f g a =
    DaWrappa (f a) (g a)


instance (Functor f, Functor g) => Functor (Parappa f g) where
    fmap f (DaWrappa fa ga) =  DaWrappa (fmap f fa) (fmap f ga)

7. Don’t ask for more typeclass instances than you need. You can let GHC tell you what to do.

```haskell
data IgnoreOne f g a b =
    IgnoringSomething (f a) (g b)
```

In [32]:
data IgnoreOne f g a b =
    IgnoringSomething (f a) (g b)


instance (Functor g) => Functor (IgnoreOne f g a) where
    fmap f (IgnoringSomething fa gb) = IgnoringSomething fa (fmap f gb)

8.
```haskell
data Notorious g o a t =
    Notorious (g o) (g a) (g t)
```

In [36]:
data Notorious g o a t =
    Notorious (g o) (g a) (g t)

instance Functor g => Functor (Notorious g o a) where
    fmap f (Notorious go ga gt) = Notorious go ga (fmap f gt)

9. You’ll need to use recursion.
```haskell
data List a =
    Nil
  | Cons a (List a)
```

In [37]:
data List a =
    Nil
  | Cons a (List a)

instance Functor List where
    fmap _ Nil = Nil
    fmap f (Cons x xs) = Cons (f x) (fmap f xs)

10. A tree of goats forms a Goat-Lord, fearsome poly-creature.

```haskell
data GoatLord a =
    NoGoat
  | OneGoat a
  | MoreGoats (GoatLord a) (GoatLord a) (GoatLord a)
```

In [ ]:
data GoatLord a =
    NoGoat
  | OneGoat a
  | MoreGoats (GoatLord a) (GoatLord a) (GoatLord a)


instance Functor GoatLord where
    fmap _ NoGoat = NoGoat
    fmap f (OneGoat x) = OneGoat (f x)
    fmap f (MoreGoats g1 g2 g3) =
        MoreGoats (fmap f g1) (fmap f g2) (fmap f g3)

11. You’ll use an extra functor for this one, although your solution might do it monomorphically without using fmap. Keep in mind that you will probably not be able to validate this one in the usual manner. Do your best to make it work.

```haskell
data TalkToMe a =
    Halt
  | Print String a
  | Read (String -> a)
```

In [ ]:
data TalkToMe a =
    Halt
  | Print String a
  | Read (String -> a)


instance Functor TalkToMe where
    fmap _ Halt = Halt
    fmap f (Print s x) = Print s (f x)
    fmap f (Read g) = Read (fmap f g)